In [85]:
#imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline, interp1d
import requests
import xml.etree.ElementTree as ET
import datetime
import math
import re
import requests
import xml.etree.ElementTree as ET

from scipy import interpolate

# Time to maturity: calculated by dividing the number of minutes until expiration of the selected options (rounded down to the nearest minute) by the number of minutes in a year.

In [86]:
def time_to_expiration(term = True):
    """
    Calculates the time to expiration.
    :param <term>: boolean ; True == "Near-term", False == "Next-term"
    """
    #current time + tomorrow
    now = datetime.datetime(2023, 4, 15, 22, 55, 8, 925690)
    day = datetime.date(2023, 4, 15)
    tomorrow = datetime.datetime(now.year, now.month, now.day) + datetime.timedelta(1)
    
    #near- or next-term
    if term == True:
        val = 0
    else:
        val = 7
    
    #calculation of minutes remaining until midnight of the current day
    minutes_to_midnight_today = int(round(abs(tomorrow - now).seconds / 60,0))
    
    #calculation of total minutes in the days between current day and expiration day
    for index in range(24,38):
        day = datetime.datetime(now.year, now.month, now.day) + datetime.timedelta(index)   #加了日期后对应的时间
        if day.weekday() == 4:      #Friday expire
            days_to_expiration = index + val - 1
            if 15 <= day.day <= 21:  # For standard SPX options expire on 8:30am, the 3rd Friday must fall in (15,21)
                minutes_to_settlement = 510
            else:
                minutes_to_settlement = 900
            break
    
    return (minutes_to_midnight_today + minutes_to_settlement + (days_to_expiration) * 24 * 60)/525600

In [87]:
(time_to_expiration(True), time_to_expiration(False))

(0.07306887366818873, 0.09224695585996956)

# YTM

In [88]:
# How to apply the boundary?
def natural_cubic_spline(time_to_maturity):
    x = [30, 60, 91, 182, 365, 730, 1095, 1825, 2555, 3650, 7300, 10950]
    for i in range(12):
        x[i] = x[i] * 24 * 60
    y = [4.53, 4.89, 5.08, 4.98, 4.65, 4, 3.75, 3.52, 3.47, 3.41, 3.74, 3.62]

    # apply natural cubic spline interpolation
    ns = CubicSpline(x, y, bc_type='natural', extrapolate=True)
    
    return ns(time_to_maturity)

In [89]:
dates = [time_to_expiration(True),time_to_expiration(False)]

estimated_risk_free_rate = []

for element in dates:
    estimated_risk_free_rate.append(float(natural_cubic_spline(element*525600))*0.01)

In [90]:
estimated_risk_free_rate

[0.04485933953514087, 0.04578557014212466]

# Determine F

Determine the option-implied forward price level, 𝐹, by identifying the options strike price at which the absolute difference between the call price and the put price is smallest. If there are multiple put-call pairs with the same minimum absolute difference value, select the lowest strike price of these pairs. 

In [91]:
def day_of_expiration():
    """
    Determines the date of near-term and next-term settlement Fridays.
    No arguments.
    """
    now = datetime.datetime(2023, 4, 15, 22, 55, 8, 925690)
    
    for index in range(24,38):
        day = datetime.datetime(now.year, now.month, now.day) + datetime.timedelta(index)
        if day.weekday() == 4:
            near_term = datetime.datetime(now.year, now.month, now.day) + datetime.timedelta(index)
            next_term = datetime.datetime(now.year, now.month, now.day) + datetime.timedelta(index+7)
            break
    
    return [near_term, next_term]   

In [92]:
day_of_expiration()

[datetime.datetime(2023, 5, 12, 0, 0), datetime.datetime(2023, 5, 19, 0, 0)]

In [93]:
near_term_option_data = pd.read_csv('C:/Users/SIPENG LI/Downloads/$spx-options-exp-2023-05-12-weekly-show-all-stacked-04-21-2023.csv')
next_term_option_data = pd.read_csv('C:/Users/SIPENG LI/Downloads/$spx-options-exp-2023-05-19-weekly-show-all-stacked-04-21-2023.csv')

In [94]:
near_term_option_data['Strike'] = np.int64(np.float64(near_term_option_data['Strike']))
next_term_option_data['Strike'] = np.int64(np.float64(next_term_option_data['Strike']))
near_term_option_data['Last'] = np.float64(near_term_option_data['Last'])
next_term_option_data['Last'] = np.float64(next_term_option_data['Last'])

In [213]:
near_data_call = near_term_option_data[near_term_option_data['IV'] == 'Call']
near_data_put = near_term_option_data[near_term_option_data['IV'] == 'Put']
next_data_call = next_term_option_data[next_term_option_data['IV'] == 'Call']
next_data_put = next_term_option_data[next_term_option_data['IV'] == 'Put']

#data_dic = {'near_data_call': near_data_call, 'near_data_put': near_data_put, 
#           'next_data_call': next_data_call, 'next_data_put': next_data_put}

At-the-money option list

In [178]:
def at_the_money_option_list(data_call, data_put):
    option_list_row_index = []
    for i in range(data_call.shape[0]):
        if ((data_call.iloc[i, 2] != 0)&(data_call.iloc[i, 4] != 0)&(data_put.iloc[i, 2] != 0)&(data_put.iloc[i, 4] != 0)&(data_call.iloc[i, 2] <= data_call.iloc[i, 4])&(data_put.iloc[i, 2] <= data_put.iloc[i, 4])):
            option_list_row_index.append(i)
    
    return option_list_row_index

In [179]:
ATM_near_data_call = near_data_call.iloc[at_the_money_option_list(near_data_call, near_data_put),:].reset_index(drop=True)
ATM_near_data_put = near_data_put.iloc[at_the_money_option_list(near_data_call, near_data_put),:].reset_index(drop=True)
ATM_next_data_call = next_data_call.iloc[at_the_money_option_list(next_data_call, next_data_put),:].reset_index(drop=True)
ATM_next_data_put = next_data_put.iloc[at_the_money_option_list(next_data_call, next_data_put),:].reset_index(drop=True)

In [182]:
# Find strike price and value difference for F
# Series with null quotes or bid price higher than ask price are not candidates to be the ATM strike
def F(data_call, data_put):
    absolute_difference = []
    for i in range(data_call.shape[0]):
        if ((data_call.iloc[i, 3] != 0)&(data_put.iloc[i, 3] != 0)&(data_call.iloc[i, 2] <= data_call.iloc[i, 4])&(data_put.iloc[i, 2] <= data_put.iloc[i, 4])):
            absolute_difference.append(abs(data_call.iloc[i, 3] - data_put.iloc[i, 3]))
        else:
            absolute_difference.append(9999)
    minimum_abs_difference = min(absolute_difference)

    strike = []
    row_index = []
    if (absolute_difference.count(minimum_abs_difference)) > 1:
        for j in range(absolute_difference.count(minimum_abs_difference)):
            row_index.append(np.where(absolute_difference == minimum_abs_difference)[0][j])
            strike.append(data_call['Strike'][row_index[j]])
        strike_price = min(strike)
        row = row_index[np.where(strike == strike_price)[0][0]]
    else:
        row = np.where(absolute_difference == minimum_abs_difference)[0][0]
        strike_price = data_call['Strike'][row]

    call_put_difference = data_call.iloc[row, 3] - data_put.iloc[row, 3]
    
    return [strike_price, call_put_difference]

In [183]:
def find_F():
    # Find forward index prices for the near- and next-term options
    dates = [time_to_expiration(True),time_to_expiration(False)]
    data_call = ATM_near_data_call
    data_put = ATM_near_data_put
    near_temp = F(data_call, data_put)
    
    data_call = ATM_next_data_call
    data_put = ATM_next_data_put
    next_temp = F(data_call, data_put)
    
    near_F = near_temp[0] + math.exp(estimated_risk_free_rate[0]*dates[0]) * near_temp[1]
    next_F = next_temp[0] + math.exp(estimated_risk_free_rate[1]*dates[1]) * next_temp[1]
    return [near_F, next_F]

In [184]:
f = find_F()
f

[4139.498358400328, 4141.506348767025]

K: the strike price equal to or otherwise immediately below the forward price, 𝐹, for 
the near- and next-term candidate constituent options. 

In [191]:
def K0():
    f = find_F()
    K = [0, 0]
    row = [0, 0]
    compare_F_and_strike = []
    # near-term K
    if (ATM_near_data_call.iloc[0, 0] - f[0]) == 0:
        K[0] = f[0]
        row[0] = 0
    else:
        for i in range(1, ATM_near_data_call.shape[0]+1):
            if (ATM_near_data_call.iloc[i, 0] - f[0]) == 0:
                K[0] = f[0]
                row[0] = i
            if (ATM_near_data_call.iloc[i, 0] - f[0]) > 0:
                K[0] = ATM_near_data_call.iloc[i-1, 0]
                row[0] = i-1
                break
    
    # Next-term K
    if (ATM_next_data_call.iloc[0, 0] - f[1]) == 0:
        K[1] = f[1]
        row[1] = 0
    else:
        for i in range(1, ATM_next_data_call.shape[0]+1):
            if (ATM_next_data_call.iloc[i, 0] - f[1]) == 0:
                K[1] = f[1]
                row[1] = i
            if (ATM_next_data_call.iloc[i, 0] - f[1]) > 0:
                K[1] = ATM_next_data_call.iloc[i-1, 0]
                row[1] = i-1
                break
    return K, row

In [192]:
K, row = K0()
K

[4135, 4140]

# Strike Selection: 

1. remove all option strikes with null quotes from both the put and the call option series
 
2. Ki list: a call if 𝐾𝑖 > 𝐾0 and a put if 𝐾𝑖 < 𝐾0; both put and call if 𝐾𝑖 = 𝐾0

In [209]:
def out_of_money_option(K, row, optiontype):
    option_list_row_index = []
    count_zero = []
    count = 0
    if optiontype == 'Put':
        for i in range(row-1, -1, -1):
            if data.iloc[i, 0] < K:
                if data.iloc[i, 2] == 0:
                    count_zero.append(0)
                    count += 1
                    if (count >= 2) & (count < row):
                        if count_zero[count-2] == 0:
                            break
                    if count == row:
                        print('VIX could not be calculated')
                else:
                    option_list_row_index.append(i)
                    count_zero.append(1)
                    
    if optiontype == 'Call':
        for i in range(row+1, data.shape[0], 1):
            if data.iloc[i, 0] > K:
                if data.iloc[i, 2] == 0:
                    count_zero.append(0)
                    count += 1
                    if count >= 2:
                        if count_zero[count-2] == 0:
                            break
                    if count == row:
                        print('VIX could not be calculated')
                else:
                    option_list_row_index.append(i)
                    count_zero.append(1)
                    
    return option_list_row_index

In [210]:
def remove_null(data_call, data_put):
    option_list_row_index = []
    for i in range(data_call.shape[0]):
        if ((data_call.iloc[i, 2] != 0)&(data_call.iloc[i, 4] != 0)&(data_put.iloc[i, 2] != 0)&(data_put.iloc[i, 4] != 0)):
            option_list_row_index.append(i)
    
    return option_list_row_index

In [214]:
near_data_call = near_data_call.iloc[remove_null(near_data_call, near_data_put),:].reset_index(drop=True)
near_data_put = near_data_put.iloc[remove_null(near_data_call, near_data_put),:].reset_index(drop=True)
next_data_call = next_data_call.iloc[remove_null(next_data_call, next_data_put),:].reset_index(drop=True)
next_data_put = next_data_put.iloc[remove_null(next_data_call, next_data_put),:].reset_index(drop=True)

In [215]:
K, row = K0()
data = near_data_call
near_term_call_option_list = out_of_money_option(K[0], row[0], 'Call')

data = near_data_put
near_term_put_option_list = out_of_money_option(K[0], row[0], 'Put')

data = next_data_call
next_term_call_option_list = out_of_money_option(K[1], row[1], 'Call')

data = next_data_put
next_term_put_option_list = out_of_money_option(K[1], row[1], 'Put')

In [216]:
near_term_call_option_list

[142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189]

In [217]:
near_term_put_option_list

[140,
 139,
 138,
 137,
 136,
 135,
 134,
 133,
 132,
 131,
 130,
 129,
 128,
 127,
 126,
 125,
 124,
 123,
 122,
 121,
 120,
 119,
 118,
 117,
 116,
 115,
 114,
 113,
 112,
 111,
 110,
 109,
 108,
 107,
 106,
 105,
 104,
 103,
 102,
 101,
 100,
 99,
 98,
 97,
 96,
 95,
 94,
 93,
 92,
 91,
 90,
 89,
 88,
 87,
 86,
 85,
 84,
 83,
 82,
 81,
 80,
 79,
 78,
 77,
 76,
 75,
 74,
 73,
 72,
 71,
 70,
 69,
 68,
 67,
 66,
 65,
 64,
 63,
 62,
 61,
 60,
 59,
 58,
 57,
 56,
 55,
 54,
 53,
 52,
 51,
 50,
 49,
 48,
 47,
 46,
 45,
 44,
 43,
 42,
 41,
 40,
 39,
 38,
 37,
 36,
 35,
 34,
 33,
 32,
 31,
 30,
 29,
 28,
 27,
 26,
 25,
 24,
 23,
 22,
 21,
 20,
 19,
 18,
 17,
 16,
 15,
 14,
 13,
 12,
 11,
 10,
 9,
 8,
 7,
 6,
 5,
 4,
 3,
 2,
 1,
 0]

In [218]:
next_term_call_option_list

[207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 280,
 281]

In [219]:
next_term_put_option_list

[205,
 204,
 203,
 202,
 201,
 200,
 199,
 198,
 197,
 196,
 195,
 194,
 193,
 192,
 191,
 190,
 189,
 188,
 187,
 186,
 185,
 184,
 183,
 182,
 181,
 180,
 179,
 178,
 177,
 176,
 175,
 174,
 173,
 172,
 171,
 170,
 169,
 168,
 167,
 166,
 165,
 164,
 163,
 162,
 161,
 160,
 159,
 158,
 157,
 156,
 155,
 154,
 153,
 152,
 151,
 150,
 149,
 148,
 147,
 146,
 145,
 144,
 143,
 142,
 141,
 140,
 139,
 138,
 137,
 136,
 135,
 134,
 133,
 132,
 131,
 130,
 129,
 128,
 127,
 126,
 125,
 124,
 123,
 122,
 121,
 120,
 119,
 118,
 117,
 116,
 115,
 114,
 113,
 112,
 111,
 110,
 109,
 108,
 107,
 106,
 105,
 104,
 103,
 102,
 101,
 100,
 99,
 98,
 97,
 96,
 95,
 94,
 93,
 92,
 91,
 90,
 89,
 88,
 87,
 86,
 85,
 84,
 83,
 82,
 81,
 80,
 79,
 78,
 77,
 76,
 75,
 74,
 73,
 72,
 71,
 70,
 69,
 68,
 67,
 66,
 65,
 64,
 63,
 62,
 61,
 60,
 59,
 58,
 57,
 56,
 55,
 54,
 53,
 52,
 51,
 50,
 49,
 48,
 47,
 46,
 45,
 44,
 43,
 42,
 41,
 40,
 39,
 38,
 37,
 36,
 35,
 34,
 33,
 32,
 31,
 30,
 29,
 28,
 27,

# Calculate volatility for both near- and next-term options

Δ𝐾𝑖 = • Highest OTM Strike 𝐾𝑖: 𝐾𝑖 −𝐾𝑖−1

     • Lowest OTM Strike 𝐾𝑖: 𝐾𝑖+1 − 𝐾𝑖
     
     • Otherwise: (𝐾𝑖+1 − 𝐾𝑖−1) / 2

Q(𝐾𝑖): Option price of the OTM option with strike 𝐾𝑖; Q(𝐾0) is the average of the 𝐾0 put option price and 𝐾0 call option price

In [229]:
def calculate_volatility(T, R, K0_index, F, near_next_type):
    if near_next_type == 'near':
        call_option_list = near_term_call_option_list
        put_option_list = near_term_put_option_list
        data_call = near_data_call
        data_put = near_data_put
    if near_next_type == 'next':
        call_option_list = next_term_call_option_list
        put_option_list = next_term_put_option_list
        data_call = next_data_call
        data_put = next_data_put
        
    # Highest OTM Strike price should be the last one of call_option_list
    # Lowest OTM strike price should be the first one of put_option_list
    
    I = 0  # sum 
    
    # First revert the order of put_option_list
    put_option_list = np.int64(np.sort(put_option_list))
    for i in range(len(put_option_list)):
        if i == 0:
            delta_K = data_put.iloc[i+1, 0] - data_put.iloc[i, 0]
            I += delta_K / np.power(data_put.iloc[i, 0], 2) * np.exp(R * T) * data_put.iloc[i, 3]
        else:
            delta_K = (data_put.iloc[i+1, 0] - data_put.iloc[i-1, 0])/2
            I += delta_K / np.power(data_put.iloc[i, 0], 2) * np.exp(R * T) * data_put.iloc[i, 3]
    
    for j in call_option_list:
        if j == max(call_option_list):
            delta_K = data_call.iloc[j, 0] - data_call.iloc[j-1, 0]
            I += delta_K / np.power(data_call.iloc[j, 0], 2) * np.exp(R * T) * data_call.iloc[i, 3]
        else:
            delta_K = (data_call.iloc[j+1, 0] - data_call.iloc[j-1, 0])/2
            I += delta_K / np.power(data_call.iloc[j, 0], 2) * np.exp(R * T) * data_call.iloc[i, 3]
    
    # For K0
    delta_K = (data_call.iloc[K0_index+1, 0] - data_call.iloc[K0_index-1, 0])/2
    Q_K0 = (data_call.iloc[K0_index, 3] + data_put.iloc[K0_index, 3])/2
    I += delta_K / np.power(data_call.iloc[K0_index, 0], 2) * np.exp(R * T) * Q_K0
    
    volatility = 2/T * I - 1/T * np.power((F/data_call.iloc[K0_index, 0]-1), 2)
    
    return volatility

In [230]:
f = find_F()
f

[4139.498358400328, 4141.506348767025]

In [231]:
near_term_volatility = calculate_volatility(dates[0], estimated_risk_free_rate[0], row[0], f[0], 'near')
next_term_volatility = calculate_volatility(dates[1], estimated_risk_free_rate[1], row[1], f[1], 'next')

In [232]:
(near_term_volatility, next_term_volatility)

(0.059630177133687984, 0.057850584594974115)

# Calculate VIX index

𝑀𝑇1: The number of minutes until expiration of the near-term options

𝑀𝑇2: The number of minutes until expiration of the next-term options

𝑀CM: The number of minutes in the given constant maturity term (30 x 24 x 60 = 43200)

𝑀365: The number of minutes in a 365-day year (365 x 1,440 = 525,600)

𝑇𝑖: 𝑀𝑇𝑖 ⁄ 𝑀365

In [233]:
MT1 = time_to_expiration(True) * 525600
MT2 = time_to_expiration(False) * 525600
MCM = 43200
M365 = 525600
T1 = MT1/M365
T2 = MT2/M365

In [234]:
volatility_index = 100 * np.sqrt((T1 * near_term_volatility * (MT2-MCM)/(MT2-MT1) + T2 * next_term_volatility * (MCM-MT1)/(MT2-MT1))*M365/MCM)

In [235]:
volatility_index

24.223969656325